In [1]:
# %load TestHarness
debugging = False
debugging = True

debugging2 = False

logging = True

def printf(f, *args):
    if len(args): 
        print(f.format(*args))
    else:
        print(f)   

def dbg(f, *args):
    if debugging:
        if len(args):
            printf('  DBG:' + f, *args)
        else:
            print(f)

def dbg2(f, *args):
    if debugging2:
        if len(args):
            printf('  DBG2:' + f, *args)
        else:
            print(f)
        
def log(f, *args):
    if logging:
        printf(f, *args)
        
def log_error(f, *args):
    if logging:
        if len(args):
            printf('*** ERROR:' + f, *args)
        else:
            print('*** ERROR:', f)
        
def class_name(instance):
    return type(instance).__name__

#------------------------------------------------------------------------------   
import time
from datetime import timedelta

#------------------------------------------------------------------------------
class TestCase(object):
    def __init__(self, name, method, inputs, expected, catchExceptions=False):
        self.name = name
        self.method = method
        self.inputs = inputs
        self.expected = expected
        self.catchExceptions = catchExceptions
        
    def run(self):
        if self.catchExceptions:
            try:
                return self.method(*self.inputs)
            except Exception as x:
                return x
        else:
                return self.method(*self.inputs)

#------------------------------------------------------------------------------
class TestSet(object):
    def __init__(self, cases):
        self.cases = cases
    
    def run_tests(self, repeat=1):
        count = 0
        errors = 0
        total_time = 0
        for case in self.cases:
            count += 1
            start_time = time.time()
            for iteration in range(repeat):
                dbg2("*** Running '{0}' iteration {1}", case.name, iteration+1)
                result = case.run()
            elapsed_time = time.time() - start_time
            total_time += elapsed_time
            if callable(case.expected):
                if not case.expected(result):
                    errors += 1
                    log_error("Test {0} failed. Returned {1}", case.name, result)
            elif result != case.expected:
                errors += 1
                log_error('Test {0} failed. Returned "{1}", expected "{2}"', case.name, result, case.expected)
        if errors:
            log_error("Tests passed: {0}; Failures: {1}", count-errors, errors)
        else:
            log("All {0} tests passed.", count)
        log("Elapsed test time: {0}", timedelta(seconds=total_time))

#------------------------------------------------------------------------------
def elapsed_time(func, *args, msg=''):
    ''' Display the elapsed time of the function.
        Return the function value.
    '''
    stime = time.time()
    result = func(*args)
    etime = time.time() - stime
    log(msg + "Elapsed test time: {0}", timedelta(seconds=etime))
    return result
        


In [2]:
elapsed_time(int, 1, msg='For a single int call: ')

For a single int call: Elapsed test time: 0:00:00.000004


1

In [6]:
def solve_tower(n, s, i, d, memo=None, dynamic=True, iterations=None, stats=True):
    ''' 
    Print the solution to the Tower's of Hanoi problem,
    where n is the number of disks, and s is the source
    pole (which contains n disks), and d is the destination
    pole which will contain n disks on completion.
    
    Returns the list of moves required for the solution.
    '''
    self = solve_tower
    key = (n, s, i, d)
    if iterations is None: 
        iterations=[0]
    else:
        iterations[0] += 1    
    
    if dynamic:
        if memo is not None: # WHOOOP! WHOOP! Do not use 'if memo:'!!!
            if key in memo:
                return memo[key]
        else:
            memo = {} # Use a dictionary of prior solutions to speed solve.
            # NOTE! An empty dictionary is False, otherwise True.

    def move(n, s, d):
        m = "- move Disk#{0} from {1} to {2}".format(n, s, d)
        return [m]

    moves = []    
    if n < 3:
        if n < 2:
            # Just because pathological test cases are possible.
            if n>0: moves += move(1, s, d)
        else:           
            # For the base case, have just two disks to move,
            # so the answer is a simple 3 move shuffle.
            moves += move(1, s, i)
            moves += move(2, s, d)
            moves += move(1, i, d)
    else:
        # Move n-1 disks to the intermediate pole.
        moves += self(n-1, s, d, i, memo, dynamic, iterations, False)

        # Assume that i contains n-1 disks, and s has the
        # last big disk, so the answer is obvious.
        moves += move(n, s, d)

        # Move the n-1 intermediate disks to the desitination.
        moves += self(n-1, i, s, d, memo, dynamic, iterations, False)
        
    if dynamic: 
        memo[key] = moves
        #if stats: log('Memo: {0}', [ (k, memo[k]) for k in sorted(memo.keys()) ] )
    if stats: log('Iterations: {0}', iterations)
    return moves

In [7]:
def solve_tower_static(n, s, i, d):
    return solve_tower(n, s, i, d, dynamic=False)

In [8]:
elapsed_time(solve_tower_static, 5, 1, 2, 3, msg='Solving Towers of Hanoi - STATIC: ')
elapsed_time(solve_tower, 5, 1, 2, 3, msg='Solving Towers of Hanoi - dynamic: ')

Iterations: [14]
Solving Towers of Hanoi - STATIC: Elapsed test time: 0:00:00.001398
Iterations: [12]
Solving Towers of Hanoi - dynamic: Elapsed test time: 0:00:00.000477


['- move Disk#1 from 1 to 3',
 '- move Disk#2 from 1 to 2',
 '- move Disk#1 from 3 to 2',
 '- move Disk#3 from 1 to 3',
 '- move Disk#1 from 2 to 1',
 '- move Disk#2 from 2 to 3',
 '- move Disk#1 from 1 to 3',
 '- move Disk#4 from 1 to 2',
 '- move Disk#1 from 3 to 2',
 '- move Disk#2 from 3 to 1',
 '- move Disk#1 from 2 to 1',
 '- move Disk#3 from 3 to 2',
 '- move Disk#1 from 1 to 3',
 '- move Disk#2 from 1 to 2',
 '- move Disk#1 from 3 to 2',
 '- move Disk#5 from 1 to 3',
 '- move Disk#1 from 2 to 1',
 '- move Disk#2 from 2 to 3',
 '- move Disk#1 from 1 to 3',
 '- move Disk#3 from 2 to 1',
 '- move Disk#1 from 3 to 2',
 '- move Disk#2 from 3 to 1',
 '- move Disk#1 from 2 to 1',
 '- move Disk#4 from 2 to 3',
 '- move Disk#1 from 1 to 3',
 '- move Disk#2 from 1 to 2',
 '- move Disk#1 from 3 to 2',
 '- move Disk#3 from 1 to 3',
 '- move Disk#1 from 2 to 1',
 '- move Disk#2 from 2 to 3',
 '- move Disk#1 from 1 to 3']

In [9]:
n=20
slow = len(elapsed_time(solve_tower_static, n, 1, 2, 3, msg='Solving Towers of Hanoi - STATIC: '))
fast = len(elapsed_time(solve_tower, n, 1, 2, 3, msg='Solving Towers of Hanoi - dynamic: '))
slow == fast

Iterations: [524286]
Solving Towers of Hanoi - STATIC: Elapsed test time: 0:00:03.433122
Iterations: [102]
Solving Towers of Hanoi - dynamic: Elapsed test time: 0:00:00.043234


True

In [ ]:
a = elapsed_time(solve_tower, 25, 1, 2, 3)
len(a)

In [ ]:
a = elapsed_time(solve_tower, 26, 1, 2, 3)
len(a)

In [ ]:
n = 0; a=[]; v='x';
while n<10: 
    n += 1
    v = input('Gimme {0}:'.format(n))
    if not v: break       
    a.append(v)
a

In [ ]:
import fileinput
import sys
log('Feed me')
for line in sys.stdin:
#for line in fileinput.input():
    log('The line is : "{0}"', line)
log('- Eat me')